In [10]:
from ase.build import bulk
from gpaw import GPAW, PW, FermiDirac
import numpy as np
from ase.units import Hartree

# Perform standard ground state calculation (with plane wave basis)
si = bulk('Si', 'diamond', 5.43)
si.center()


calc = GPAW(mode=PW(400),xc='PBE',setups='sg15',
            kpts=(16, 16, 16),symmetry='off',
            occupations=FermiDirac(0.0001))
si.calc = calc
si.get_potential_energy()
calc.write('Si_gs.gpw')


  ___ ___ ___ _ _ _  
 |   |   |_  | | | | 
 | | | | | . | | | | 
 |__ |  _|___|_____|  1.5.1
 |___|_|             

User:   drg@forsite
Date:   Wed May  8 14:11:29 2019
Arch:   x86_64
Pid:    24060
Python: 3.6.7
gpaw:   /usr/local/lib/python3.6/dist-packages/gpaw
_gpaw:  /usr/local/lib/python3.6/dist-packages/
        _gpaw.cpython-36m-x86_64-linux-gnu.so
ase:    /home/drg/.local/lib/python3.6/site-packages/ase (version 3.17.0)
numpy:  /home/drg/.local/lib/python3.6/site-packages/numpy (version 1.16.3)
scipy:  /home/drg/.local/lib/python3.6/site-packages/scipy (version 1.2.1)
units:  Angstrom and eV
cores:  1

Input parameters:
  kpts: [16 16 16]
  mode: {ecut: 400.0,
         gammacentered: False,
         name: pw}
  occupations: {name: fermi-dirac,
                width: 0.0001}
  setups: sg15
  symmetry: off
  xc: PBE

System changes: positions, numbers, cell, pbc, initial_charges, initial_magmoms 

Initialize ...

Norm-conserving UPF setup:
  Element:   Si
  Z:         14
  Vale

In [11]:
from TimeDependentHamiltonian import TimeDependentHamiltonian
from TimeDependentPropagator import TimeDependentPropagator
import matplotlib.pyplot as plt
%matplotlib inline
TDH=TimeDependentHamiltonian(calc)
TDP=TimeDependentPropagator(TDH)

Timing:                              incl.     excl.
-----------------------------------------------------------
Hamiltonian:                         0.016     0.000   0.0% |
 Atomic:                             0.000     0.000   0.0% |
  XC Correction:                     0.000     0.000   0.0% |
 Calculate atomic Hamiltonians:      0.000     0.000   0.0% |
 Communicate:                        0.000     0.000   0.0% |
 Initialize Hamiltonian:             0.000     0.000   0.0% |
 Poisson:                            0.000     0.000   0.0% |
 XC 3D grid:                         0.015     0.015   0.0% |
LCAO initialization:                 1.236     0.017   0.0% |
 LCAO eigensolver:                   0.292     0.002   0.0% |
  Calculate projections:             0.001     0.001   0.0% |
  DenseAtomicCorrection:             0.004     0.004   0.0% |
  Distribute overlap matrix:         0.001     0.001   0.0% |
  Orbital Layouts:                   0.011     0.011   0.0% |
  Potential matrix:

In [12]:
epsilon,freq=TDP.linear_response(dt=1,steps=1000)

 43%|████▎     | 426/1000 [25:49<33:19,  3.48s/it]  

KeyboardInterrupt: 

In [ ]:
J=TDP.J[:,2]
time=np.arange(J.size)/41.341 

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(time,-J)
plt.grid()
plt.xlim([0,24])
plt.xticks(np.arange(0,25,2),fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('$t$,fs',fontsize=20)
plt.ylabel('$J(t)$,a.u.',fontsize=20)
plt.savefig('current.png',dpi=600)

In [ ]:
epsilon.real[int(freq.size/2)]=0
epsilon.imag[int(freq.size/2)]=0

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
plt.figure(figsize=(8,8))
plt.plot(freq*Hartree,gaussian_filter1d(epsilon.real,6),label='Re$\epsilon(\omega)$')
plt.plot(freq*Hartree,gaussian_filter1d(epsilon.imag,6),label='Im$\epsilon(\omega)$')


plt.legend(fontsize=20,loc='best')
plt.xlabel('$\omega$,eV',fontsize=20)
plt.ylabel('$\epsilon(\omega)$',fontsize=20)
plt.xticks(np.arange(-10,11),fontsize=14)
plt.yticks(np.arange(-100,110,10),fontsize=14)
plt.grid()
plt.xlim([0,10])
plt.ylim([-20,60])
plt.savefig('epsilon_gauss.png',dpi=600)